# feature engineering

In [160]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/train.csv')

In [161]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


## Fuel type

In [166]:
df["fuel_type"].unique()

array(['Gasoline', 'E85 Flex Fuel', nan, 'Hybrid', 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [167]:
df.groupby("fuel_type").size()

fuel_type
Diesel              3955
E85 Flex Fuel       5406
Gasoline          165940
Hybrid              6832
Plug-In Hybrid       521
not supported         15
–                    781
dtype: int64

In [168]:
df.groupby("fuel_type")["price"].mean()

fuel_type
Diesel            48769.587358
E85 Flex Fuel     26729.755457
Gasoline          43360.013782
Hybrid            55473.451698
Plug-In Hybrid    44150.967370
not supported     20692.733333
–                 36414.285531
Name: price, dtype: float64

In [173]:
# df["fuel_type"]の'not supported'を抜き出しで観察したい
# fuel_typeとengineの行だけ
df[(df["fuel_type"] == 'not supported') | (df["fuel_type"] == '–')][["fuel_type", "engine"]]


,fuel_type,engine
532,–,–
1155,–,–
1307,–,–
1362,–,–
1741,–,–
...,...,...
187399,–,–
187495,–,284.0HP 3.5L V6 Cylinder Engine Gasoline Fuel
187664,–,–
187820,–,–


In [174]:
# fuel_typeが'not supported'、null、または'–'の行をフィルタリング
no_fuel = df[(df["fuel_type"] == 'not supported') | (df["fuel_type"].isnull()) | (df["fuel_type"] == '–')]

# engine列から正規表現でfuel_typeを抽出
extracted_fuel_types = no_fuel["engine"].str.extract(r"(\w+) Fuel")[0]

# 元のデータフレームの対応するfuel_type列に代入
df.loc[(df["fuel_type"] == 'not supported') | (df["fuel_type"].isnull()) | (df["fuel_type"] == '–'), "fuel_type"] = extracted_fuel_types

In [211]:
df.groupby("fuel_type").size()

fuel_type
Diesel             3955
E85 Flex Fuel      5406
Electric          11139
Gasoline         165135
Hybrid             1946
dtype: int64

In [209]:
# engine の中身が Electricはfuel_typeをElectricに変更
df.loc[df["engine"].str.contains("Electric"), "fuel_type"] = "Electric"
df.loc[df["fuel_type"] == 'Flex', "fuel_type"] = "E85 Flex Fuel"
df.loc[df["fuel_type"] == 'Plug-In Hybrid', "fuel_type"] = "Hybrid"

In [210]:
df["fuel_type"].unique()

array(['Gasoline', 'E85 Flex Fuel', 'Electric', 'Diesel', nan, 'Hybrid'],
      dtype=object)

In [220]:
df[df["fuel_type"].isnull()][["fuel_type"]].shape[0]

952

In [222]:
df[df["fuel_type"].isnull()][["fuel_type", "engine"]]

,fuel_type,engine
51,NaN,Dual Motor - Standard
532,NaN,–
1155,NaN,–
1307,NaN,–
1362,NaN,–
...,...,...
187399,NaN,–
187664,NaN,–
187820,NaN,–
188189,NaN,–


## transmission

In [287]:
df["transmission"].unique()

array(['A/T', 'Transmission w/Dual Shift Mode', '7-Speed A/T',
       '8-Speed A/T', '10-Speed Automatic', '1-Speed A/T', '6-Speed A/T',
       '10-Speed A/T', '9-Speed A/T', '8-Speed Automatic',
       '9-Speed Automatic', '5-Speed A/T', 'Automatic',
       '7-Speed Automatic with Auto-Shift', 'CVT Transmission',
       '5-Speed M/T', 'M/T', '6-Speed M/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T', '2-Speed A/T',
       '1-Speed Automatic', 'Automatic CVT', '4-Speed A/T',
       '6-Speed Manual', 'Transmission Overdrive Switch',
       '8-Speed Automatic with Auto-Shift', '7-Speed Manual',
       '7-Speed Automatic', '9-Speed Automatic with Auto-Shift',
       '6-Speed Automatic with Auto-Shift',
       '6-Speed Electronically Controlled Automatic with O', 'F', 'CVT-F',
       '8-Speed Manual', 'Manual', '–', '2', '6 Speed At/Mt',
       '5-Speed Automatic', '2-Speed Automatic', '8-SPEED A/T', '7-Speed',
       'Variable', 'Single-Speed Fixed Gear', '8-SPEED AT',


In [288]:
# Automatic Transmission (A/T, Automatic, CVT), Manual Transmission ( M/T, Manual)
# Transmission Overdrive Switch, Electronically Controlled Automatic, F, –, 2, 6 Speed At/Mt, Variable, Single-Speed Fixed Gea, SCHEDULED FOR OR IN PRODUCTION

df['transmission'].str.extract('(A/T|Automatic|CVT)', expand=False)

0               A/T
1               A/T
2               A/T
3               NaN
4               A/T
            ...    
188528          NaN
188529          A/T
188530          A/T
188531    Automatic
188532          NaN
Name: transmission, Length: 188533, dtype: object

In [317]:
df["transmission_category"] = df['transmission']
df.loc[df['transmission'].str.contains('(A/T|Automatic|AT|CVT|Transmission Overdrive Switch|F|Variable)', na=False), "transmission_category"] = "Automatic"
df.loc[df['transmission'].str.contains('(M/T|Manual|Mt)', na=False), "transmission_category"] = "Manual"
df.loc[(df['transmission_category'] != "Automatic") & (df['transmission_category'] != "Manual"), "transmission_category"] = "Other"

/var/folders/yx/yl3qgng93hgb360mxcybjjr40000gn/T/ipykernel_42488/4071305191.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains('(A/T|Automatic|AT|CVT|Transmission Overdrive Switch|F|Variable)', na=False), "transmission_category"] = "Automatic"
/var/folders/yx/yl3qgng93hgb360mxcybjjr40000gn/T/ipykernel_42488/4071305191.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains('(M/T|Manual|Mt)', na=False), "transmission_category"] = "Manual"


In [318]:
df["transmission_category"].unique()

array(['Automatic', 'Other', 'Manual'], dtype=object)

In [327]:
df.groupby("transmission_category")["price"].mean()

transmission_category
Automatic    44065.302452
Manual       33810.487176
Other        50890.852737
Name: price, dtype: float64

In [328]:
df["price"].describe()

count    1.885330e+05
mean     4.387802e+04
std      7.881952e+04
min      2.000000e+03
25%      1.700000e+04
50%      3.082500e+04
75%      4.990000e+04
max      2.954083e+06
Name: price, dtype: float64

# Improvement model using new features

In [506]:
df = pd.read_csv('../data/train.csv')

In [507]:
def update_fuel_type(df: pd.DataFrame) -> pd.DataFrame:
    no_fuel = df[(df["fuel_type"] == 'not supported') | (df["fuel_type"].isnull()) | (df["fuel_type"] == '–')]
    extracted_fuel_types = no_fuel["engine"].str.extract(r"(\w+) Fuel")[0]
    df.loc[(df["fuel_type"] == 'not supported') | (df["fuel_type"].isnull()) | (df["fuel_type"] == '–'), "fuel_type"] = extracted_fuel_types
    df.loc[df["engine"].str.contains("Electric"), "fuel_type"] = "Electric"
    df.loc[df["fuel_type"] == 'Flex', "fuel_type"] = "E85 Flex Fuel"
    df.loc[df["fuel_type"] == 'Plug-In Hybrid', "fuel_type"] = "Hybrid"
    return df

In [508]:
def transmission_category(df: pd.DataFrame) -> pd.DataFrame:
    df["transmission_category"] = df['transmission']
    df.loc[df['transmission'].str.contains('(A/T|Automatic|AT|CVT|Transmission Overdrive Switch|F|Variable)', na=False), "transmission_category"] = "Automatic"
    df.loc[df['transmission'].str.contains('(M/T|Manual|Mt)', na=False), "transmission_category"] = "Manual"
    df.loc[(df['transmission_category'] != "Automatic") & (df['transmission_category'] != "Manual"), "transmission_category"] = "Other"
    return df

In [509]:
df = update_fuel_type(df)
df = transmission_category(df)

/var/folders/yx/yl3qgng93hgb360mxcybjjr40000gn/T/ipykernel_42488/1166916524.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains('(A/T|Automatic|AT|CVT|Transmission Overdrive Switch|F|Variable)', na=False), "transmission_category"] = "Automatic"
/var/folders/yx/yl3qgng93hgb360mxcybjjr40000gn/T/ipykernel_42488/1166916524.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains('(M/T|Manual|Mt)', na=False), "transmission_category"] = "Manual"


In [511]:
df["fuel_type"].unique()

array(['Gasoline', 'E85 Flex Fuel', 'Electric', 'Diesel', nan, 'Hybrid'],
      dtype=object)

In [510]:
df["transmission_category"].unique()

array(['Automatic', 'Other', 'Manual'], dtype=object)

In [438]:
display(df)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmission_category
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,Automatic
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,Automatic
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,Automatic
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,Other
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,Automatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,Other
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,Automatic
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,Automatic
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,Automatic


In [448]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    res = df.copy()
    res["clean_title"] = res["clean_title"].fillna("NaN")
    res["is.clean_title"] = res["clean_title"] == "Yes"
    res["age"] = (2024 - res["model_year"]).map(lambda x: max(x, 1))
    res["milage_per_year"] = res["milage"] / res["age"]
    res["had_accident"] = res["accident"] == "At least 1 accident or damage reported"
    res["avg_price"] =  res.groupby(['brand', 'model'])['price'].transform('mean')
    res["model_year"] = res["model_year"]
    res["avg_price_from_fuel_type"] = res.groupby("fuel_type")["price"].transform('mean')
    res.loc[res['fuel_type'].isnull(), "avg_price_from_fuel_type"] = res["price"].mean()
    res["trans_category"] = res["transmission_category"]
    return res

In [ ]:
from sklearn.model_selection import train_test_split
preprocessed = preprocess(df)
preprocessed = pd.get_dummies(preprocessed, columns=['trans_category'])
train, val = train_test_split(preprocessed, test_size = 0.3, random_state = 42)

In [467]:
display(preprocessed)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,transmission_category,is.clean_title,age,milage_per_year,had_accident,avg_price,avg_price_from_fuel_type,trans_category_Automatic,trans_category_Manual,trans_category_Other
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,...,Automatic,True,17,12529.411765,False,12778.315972,43258.950416,True,False,False
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,...,Automatic,True,22,6511.363636,True,11972.785714,43258.950416,True,False,False
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,...,Automatic,True,22,6215.045455,False,24054.754098,26724.668147,True,False,False
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,...,Other,True,7,2785.714286,False,48915.842105,43258.950416,False,False,True
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,...,Automatic,True,3,2462.666667,False,64855.673077,43258.950416,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,...,Other,True,7,7000.000000,False,51072.630631,43258.950416,False,False,True
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,...,Automatic,True,6,4766.666667,True,56035.168224,43258.950416,True,False,False
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,...,Automatic,True,3,4550.000000,False,64000.701754,43258.950416,True,False,False
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,...,Automatic,False,2,6947.500000,False,37505.533835,43258.950416,True,False,False


In [524]:
preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id                        188533 non-null  int64  
 1   brand                     188533 non-null  object 
 2   model                     188533 non-null  object 
 3   model_year                188533 non-null  int64  
 4   milage                    188533 non-null  int64  
 5   fuel_type                 187581 non-null  object 
 6   engine                    188533 non-null  object 
 7   transmission              188533 non-null  object 
 8   ext_col                   188533 non-null  object 
 9   int_col                   188533 non-null  object 
 10  accident                  186081 non-null  object 
 11  clean_title               188533 non-null  object 
 12  price                     188533 non-null  int64  
 13  transmission_category     188533 non-null  o

In [469]:
X_train = train[["is.clean_title", "milage_per_year", "had_accident", "avg_price", "model_year", "avg_price_from_fuel_type", "trans_category_Automatic", "trans_category_Manual", "trans_category_Other"]]
y_train = train["price"]

In [470]:
from sklearn.ensemble import RandomForestRegressor

In [471]:
forest = RandomForestRegressor(n_estimators = 10, max_depth = 5, random_state = 42)
forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, n_estimators=10, random_state=42)

In [472]:
X_val = val[["is.clean_title", "milage_per_year", "had_accident", "avg_price", "model_year", "avg_price_from_fuel_type", "trans_category_Automatic", "trans_category_Manual", "trans_category_Other"]]
y_val = val["price"]

In [473]:
y_pred = forest.predict(X_val)
result = pd.DataFrame({"y_val": y_val, "y_pred": y_pred})
result["diff"] = result["y_val"] - result["y_pred"]

In [474]:
result

,y_val,y_pred,diff
111355,37999,50119.145750,-12120.145750
182258,30000,48399.155684,-18399.155684
14147,26772,42440.518216,-15668.518216
79313,24999,27415.501279,-2416.501279
101160,47995,51410.684068,-3415.684068
...,...,...,...
95824,26500,42440.518216,-15940.518216
183599,54500,50319.036522,4180.963478
10343,10000,13162.981905,-3162.981905
76346,22989,30247.831867,-7258.831867


In [475]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [476]:
np.sqrt(mean_squared_error(y_val, y_pred))

np.float64(66422.39425549973)

In [477]:
# 特徴量重要度の取得
importances = forest.feature_importances_

# 特徴量名の取得
feature_names = X_train.columns

# 特徴量重要度のデータフレームの作成
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})

# 重要度でソート
importance_df = importance_df.sort_values(by='importance', ascending=False)

In [478]:
print(importance_df)

                    feature  importance
3                 avg_price    0.629794
4                model_year    0.196419
1           milage_per_year    0.146705
0            is.clean_title    0.011004
6  trans_category_Automatic    0.006695
2              had_accident    0.003423
5  avg_price_from_fuel_type    0.003006
8      trans_category_Other    0.002954
7     trans_category_Manual    0.000000


# Submission

In [566]:
test = pd.read_csv('../data/test.csv')

In [567]:
def get_avg_price_by_fuel_type(df: pd.DataFrame) -> dict:
    return df.groupby("fuel_type")["price"].mean().to_dict()

def get_avg_price_from_overall_fuel_type(df: pd.DataFrame) -> dict:
    return df["price"].mean()

In [568]:
def get_avg_price_from_brand_model(df: pd.DataFrame) -> dict:
    return df.groupby(['brand', 'model'])['avg_price'].mean().to_dict()

def get_avg_price_from_brand(df: pd.DataFrame) -> dict:
    return df.groupby('brand')['avg_price'].mean().to_dict()

In [569]:
avg_price_from_brand_model = get_avg_price_from_brand_model(preprocessed)

avg_price_from_brand = get_avg_price_from_brand(preprocessed)

avg_price_by_fuel_type = get_avg_price_by_fuel_type(preprocessed)

avg_price_from_overall_fuel_type = get_avg_price_from_overall_fuel_type(preprocessed)

In [570]:
def retrieve_value_avg_price(brand: str, model: str, from_brand_model: dict, from_brand: dict) -> float:
    """
    Retrieve a value from `avg_price` dictionary using `brand` and `model` as keys.
    If `model` is not found in the dictionary, use `brand` to retrieve the value.
    If `brand` key is not found, return None.
    """
    if (brand, model) in from_brand_model:
        return from_brand_model[(brand, model)]
    if brand in from_brand:
        return from_brand[brand]

    return None

In [571]:
def retrieve_value_avg_price_from_fuel_type(fuel_type: str, from_fuel_type: dict, from_overall: float) -> float:
    """
    Retrieve a value from `avg_price` dictionary using `fuel_type` as a key.
    If `fuel_type` is nan, return the value from the overall average.
    """
    if fuel_type in from_fuel_type:
        return from_fuel_type[fuel_type]
    if pd.isnull(fuel_type):
        return from_overall
    

    return from_overall

In [572]:
def preprocess_test(df: pd.DataFrame) -> pd.DataFrame:
    res = df.copy()
    res["clean_title"] = res["clean_title"].fillna("NaN")
    res["is.clean_title"] = res["clean_title"] == "Yes"
    res["age"] = (2024 - res["model_year"]).map(lambda x: max(x, 1))
    res["milage_per_year"] = res["milage"] / res["age"]
    res["had_accident"] = res["accident"] == "At least 1 accident or damage reported"
    res["model_year"] = res["model_year"]
    res["avg_price"] =  res.apply(lambda x: retrieve_value_avg_price(x['brand'], x['model'], avg_price_from_brand_model, avg_price_from_brand), axis=1)
    res["avg_price_from_fuel_type"] = res.apply(lambda x: retrieve_value_avg_price_from_fuel_type(x['fuel_type'], avg_price_by_fuel_type, avg_price_from_overall_fuel_type), axis=1)
    res["trans_category"] = res["transmission_category"]
    return res

In [573]:
test = transmission_category(test)
test = preprocess_test(test)
test = pd.get_dummies(test, columns=['trans_category'])

/var/folders/yx/yl3qgng93hgb360mxcybjjr40000gn/T/ipykernel_42488/1166916524.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains('(A/T|Automatic|AT|CVT|Transmission Overdrive Switch|F|Variable)', na=False), "transmission_category"] = "Automatic"
/var/folders/yx/yl3qgng93hgb360mxcybjjr40000gn/T/ipykernel_42488/1166916524.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df['transmission'].str.contains('(M/T|Manual|Mt)', na=False), "transmission_category"] = "Manual"


In [574]:
display(test)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,...,transmission_category,is.clean_title,age,milage_per_year,had_accident,avg_price,avg_price_from_fuel_type,trans_category_Automatic,trans_category_Manual,trans_category_Other
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,...,Automatic,True,9,10888.888889,False,19582.380000,43258.950416,True,False,False
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,...,Automatic,True,4,2285.500000,False,74167.477612,56718.459404,True,False,False
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,...,Automatic,False,2,14060.500000,False,50420.818345,43258.950416,True,False,False
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,...,Automatic,False,8,7657.250000,False,28007.062500,43258.950416,True,False,False
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,...,Automatic,True,6,9833.333333,False,24829.909483,43258.950416,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125685,314218,Mercedes-Benz,GL-Class GL 450 4MATIC,2014,83315,Gasoline,362.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Black,...,Automatic,True,10,8331.500000,False,35665.622691,43258.950416,True,False,False
125686,314219,Audi,Q7 55 Prestige,2019,29336,Gasoline,3.0 Liter Turbo,Automatic,White,Black,...,Automatic,False,5,5867.200000,False,56878.435714,43258.950416,True,False,False
125687,314220,Audi,A6 3.0T Premium Plus,2012,77634,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,...,Automatic,True,12,6469.500000,False,29365.696203,43258.950416,True,False,False
125688,314221,Audi,Q7 3.0T Premium,2012,112000,Gasoline,333.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Black,Black,...,Automatic,True,12,9333.333333,False,33872.352941,43258.950416,True,False,False


In [575]:
X_test = test[["is.clean_title", "milage_per_year", "had_accident", "avg_price", "model_year", "avg_price_from_fuel_type", "trans_category_Automatic", "trans_category_Manual", "trans_category_Other"]]

In [577]:
y_submit = forest.predict(X_test)
test["price"] = y_submit

In [578]:
test[["id", "price"]].to_csv("../submission/20240911-random-forest_addFeature.csv", index = False)
